In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.io import read_image
from PIL import Image

In [ ]:
import os
import pandas as pd
import shutil
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.io import read_image

import torch.nn as nn
import torch.nn.functional as F


In [ ]:
#class LabelTransform():
#    def __init__(self):
#        df = pd.read_csv('../Training/data/annotations_train_file.csv')
#        self.num_classes = len(df['label'].unique())
#    
#    def transform(self, labels):
#        labels_tensor = torch.tensor(labels)
#        one_hot = F.one_hot(labels_tensor, num_classes=self.num_classes)
#        return one_hot.type(torch.float32)

def label_transform(labels):
    labels_tensor = torch.tensor(labels)
    one_hot = F.one_hot(labels_tensor, num_classes=4271)
    return one_hot.type(torch.float32)

class PlantImageDataset(torch.utils.data.Dataset):
    """For working with plant images from the inaturalist dataset"""
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        #label = self.img_labels.iloc[idx, 1]
        label = self.img_labels.iloc[idx, 1:]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        #self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc1 = nn.Linear(44944, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4271)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x.unsqueeze(1)

In [ ]:
transform = transforms.Compose(
    [transforms.Resize(256,antialias=True),
    transforms.CenterCrop(224),
    transforms.ConvertImageDtype(torch.float32),
    #transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
#label_transform = LabelTransform()

In [ ]:
#label_transform.num_classes

In [ ]:
train_data = PlantImageDataset(annotations_file="data/annotations_train_file.csv",
                         img_dir="data/2021_train_mini/",
                         transform=transform,
                         target_transform=label_transform
                              )

val_data = PlantImageDataset(annotations_file="../Training/data/annotations_val_file.csv",
                         img_dir="../Training/data/2021_train_mini/",
                         transform=transform,
                         target_transform=label_transform
                            )

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=False, num_workers=1)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=4, shuffle=False, num_workers=1)

In [ ]:
X, y = next(iter(train_dataloader))
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

In [ ]:
image, label = next(iter(train_data))

In [ ]:
label.argmax()

In [ ]:
image.shape